<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

# Web Scraping

_Author: Dave Yerrington (SF)_

---

## Learning Objectives
- Revisit how to locate elements on a webpage
- Aquire unstructure data from the internet using Beautiful soup.
- Discuss limitations associated with simple requests and urllib libraries
- Introduce Selenium as a solution, and implement a scraper using selenium

## Lesson Guide

- [Introduction](#intro)
- [Building a web scraper](#building-scraper)
- [Retrieving data from the HTML page](#retrieving-data)
    - [Retrieving the restaurant names](#retrieving-names)
    - [Challenge: Retrieving the restaurant locations](#retrieving-locations)
    - [Retrieving the restaurant prices](#retrieving-prices)


- [Summary](#summary)

<a id="intro"></a>
## Introduction

In this codealong lesson, we'll build a web scraper using requests and BeautifulSoup. We will also explore how to use a headless browser called Selenium.

We'll begin by scraping OpenTable's DC listings. We're interested in knowing the restaurant's **name, location, price, and how many people booked it today.**

OpenTable provides all of this information on this given page: http://www.opentable.com/washington-dc-restaurant-listings

Let's inspect the elements of this page to assure we can find each of the bits of information in which we're interested.

---

<a id="building-scraper"></a>
## Building a web scraper

Now, let's build a web scraper for OpenTable using urllib and Beautiful Soup:

In [1]:
! pip install bs4

  Using cached bs4-0.0.1.tar.gz (1.1 kB)
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1279 sha256=0fa10260c07edd5417ffba5a2290537b0bd47d15d4236ba78e406525dee37406
  Stored in directory: c:\users\13305\appdata\local\pip\cache\wheels\0a\9e\ba\20e5bbc1afef3a491f0b3bb74d508f99403aabe76eda2167ca
Successfully built bs4


In [13]:
# import our necessary first packages
from bs4 import BeautifulSoup
import requests

In [3]:
# set the url we want to visit
url = "http://www.opentable.com/washington-dc-restaurant-listings"

# visit that url, and grab the html of said page
html = requests.get(url)

At this point, what is in html?

In [4]:
# .text returns the request content in Unicode
html.text[:500]

'           <!DOCTYPE html><html lang="en"><head><meta charset="utf-8"/><meta http-equiv="X-UA-Compatible" content="IE=9; IE=8; IE=7; IE=EDGE"/> <title>Restaurant Reservation Availability</title>    <meta  name="robots" content="noindex,nofollow" > </meta>     <link rel="shortcut icon" href="//components.otstatic.com/components/favicon/1.0.6/favicon/favicon.ico" type="image/x-icon"/><link rel="icon" href="//components.otstatic.com/components/favicon/1.0.6/favicon/favicon-16.png" sizes="16x16"/><l'

We will need to convert this html objct into a soup object so we can parse it using python and BS4

In [22]:
# convert this into a soup object
soup = BeautifulSoup(html.text, 'html.parser')
soup

 Washington, D.C. Area Restaurants List | OpenTable   window.oc=window.oc||{};oc.renderedComponents=oc.renderedComponents||{};oc.renderedComponents["favicon"]="1.0.6"; 

      window.OTDataLayer = [{"domain":1,"language":"en-US","metro_id":9,"page_name":"restaurantlist","user_type":"anonymous","visitor_status":"Logged Out"}];
    
window.OT = window.OT || {};
window.OT.SRS = {
  abTestingAnalytics: {"account":"2d65a94551a4a1caddd5a825a88698a89e19f2d5","hostname":"ac.opentable.com"},
  disableRaven: false,
  domainId: 1,
  jsHeadVersion: "1.0.91",
  lang: "en",
  ocUrl: window.location.protocol + '//oc-registry.opentable.com/v2/',
  requestId: "8d93ad89-0227-44d2-9f41-a33be935bf7c"
};

window.OT.searchAutoComplete = {};
window.OT.searchAutoComplete.ticketedRids = [681,1537,3334,7055,11065,29995,47119,53716,89905,100033,112846,114886,118765,140968,160504,185560,185773,210868,211918,212878,233239,252232,263128,267508,268867,270751,335149,347206,349942,984418,986635,986725];

window.oc = w

<a id="retrieving-data"></a>
### Retrieving data from the HTML page

Let's first find each restaurant name listed on the page we've loaded. How do we find the page location of the restaurant? (Hint: We need to know where in the **HTML** the restaurant element is housed.) In order to find the HTML that renders the restaurant location, we can use Google Chrome's Inspect tool:

> http://www.opentable.com/washington-dc-restaurant-listings

> 1. Visit the URL above. 

> 2. Right-click on an element you are interested in, then choose Inspect (in Chrome). 

> 3. This will open the Developer Tools and show the HTML used to render the selected page element. 

> Throughout this lesson, we will use this method to find tags associated with elements of the page we want to scrape.

See if you can find the restaurant name on the page. Keep in mind there are many restaurants loaded on the page.

In [6]:
# print the restaurant names list
soup.find_all(name='span', attrs={'class':'rest-row-name-text'})

[<span class="rest-row-name-text">Sheila Station</span>,
 <span class="rest-row-name-text">Breannes</span>,
 <span class="rest-row-name-text">Kertzmann Forges</span>,
 <span class="rest-row-name-text">Brooklyn Cummings</span>,
 <span class="rest-row-name-text">Quis Wisoky</span>,
 <span class="rest-row-name-text">Et Ford</span>,
 <span class="rest-row-name-text">Gladys Keys</span>,
 <span class="rest-row-name-text">Goldner</span>,
 <span class="rest-row-name-text">Parks</span>,
 <span class="rest-row-name-text">1286 Reilly</span>,
 <span class="rest-row-name-text">Marcos Runolfsdottir</span>,
 <span class="rest-row-name-text">Laboriosam Stark</span>,
 <span class="rest-row-name-text">Aimees</span>,
 <span class="rest-row-name-text">Vandervort</span>,
 <span class="rest-row-name-text">869 Hilll</span>,
 <span class="rest-row-name-text">Nesciunt</span>,
 <span class="rest-row-name-text">Club</span>,
 <span class="rest-row-name-text">Flavies</span>,
 <span class="rest-row-name-text">Ports

It is important to always keep in mind the data types that were returned. Note this is a `list`, and we know that immediately by observing the outer square brackets and commas separating each tag.

Next, note the elements of the list are `Tag` objects, not strings. (If they were strings, they would be surrounded by quotes.) The Beautiful Soup authors chose to display a `Tag` object visually as a text representation of the tag and its contents. However, being an object, it has many methods that we can call on it. For example, next we will use the `encode_contents()` method to return the tag's contents encoded as a Python string.

<a id="retrieving-names"></a>
#### Retrieving the restaurant names

Now that we found a list of tags containing the restaurant names, let's think how we can loop through them all one-by-one. In the following cell, we'll print out the name (and **only** the clean name, not the rest of the html) of each restaurant.

In [8]:
# for each element you find, print out the restaurant name
for entry in soup.find_all(name='span', attrs={'class':'rest-row-name-text'}):
    print(entry.text)

Sheila Station
Breannes
Kertzmann Forges
Brooklyn Cummings
Quis Wisoky
Et Ford
Gladys Keys
Goldner
Parks
1286 Reilly
Marcos Runolfsdottir
Laboriosam Stark
Aimees
Vandervort
869 Hilll
Nesciunt
Club
Flavies
Ports
Lights
Fork
Walsh
Dolores Forges
Winifred Zboncak
Simones
Pagac
Rem Walsh
Nostrum Gorczany
Repudiandae Hilpert
Carsons
Soluta Hodkiewicz
Adrien Denesik
Vilma Rohan
Odit Bins
1016 Rice
Beier
1054 McLaughlin
Mylene Forest
Rice
Aut Brakus
Haven
Omnis
Antoinette Kautzer
236 Murray
Est Rolfson
894 Gusikowski
Mills
Lakes
Kshlerin Manors
Knoll
Consectetur
Ut
Brias
Zola Springs
Eos Spencer
Jonatans
648 Prosacco
Marie Lodge
Fahey
Vel
Aut
Ipsa Prairie
Ut Williamson
Nihil
Facilis Schulist
Terry
1165 Abbott
Unde Flats
Durgan
355 Bartoletti
643 Altenwerth
Et
Architecto Track
Inlet
Odit Valley
Expressway
Et
Stream
Schowalter
Noelias
Louvenias
Consequatur Hollow
Plains
Unions
Schiller
White Flat
Keeling Extension
America Plain
Fuga
Numquam Strosin
Error Gislason
Dessies
Streich
Minus Field
Lin

Great!

<a id="retrieving-locations"></a>
#### Challenge: Retrieving the restaurant locations

Can you repeat that process for finding the location? For example, barmini by Jose Andres is in the location listed as "Penn Quarter" in our search results.

In [10]:
# first, see if you can identify the location for all elements -- print it out
for entry in soup.find_all(name='span', attrs={'class':'rest-row-name-text'}):
    print(entry.text)

Sheila Station
Breannes
Kertzmann Forges
Brooklyn Cummings
Quis Wisoky
Et Ford
Gladys Keys
Goldner
Parks
1286 Reilly
Marcos Runolfsdottir
Laboriosam Stark
Aimees
Vandervort
869 Hilll
Nesciunt
Club
Flavies
Ports
Lights
Fork
Walsh
Dolores Forges
Winifred Zboncak
Simones
Pagac
Rem Walsh
Nostrum Gorczany
Repudiandae Hilpert
Carsons
Soluta Hodkiewicz
Adrien Denesik
Vilma Rohan
Odit Bins
1016 Rice
Beier
1054 McLaughlin
Mylene Forest
Rice
Aut Brakus
Haven
Omnis
Antoinette Kautzer
236 Murray
Est Rolfson
894 Gusikowski
Mills
Lakes
Kshlerin Manors
Knoll
Consectetur
Ut
Brias
Zola Springs
Eos Spencer
Jonatans
648 Prosacco
Marie Lodge
Fahey
Vel
Aut
Ipsa Prairie
Ut Williamson
Nihil
Facilis Schulist
Terry
1165 Abbott
Unde Flats
Durgan
355 Bartoletti
643 Altenwerth
Et
Architecto Track
Inlet
Odit Valley
Expressway
Et
Stream
Schowalter
Noelias
Louvenias
Consequatur Hollow
Plains
Unions
Schiller
White Flat
Keeling Extension
America Plain
Fuga
Numquam Strosin
Error Gislason
Dessies
Streich
Minus Field
Lin

In [ ]:
# now print out EACH location for the restaurants

<a id="retrieving-prices"></a>
#### Retrieving the restaurant prices

Ok, we've figured out the restaurant name and location. Now we need to grab the price (number of dollar signs on a scale of one to four) for each restaurant. We'll follow the same process.

In [9]:
# print out all prices
soup.find_all('div', {'class':'rest-row-pricing'})

[<div class="rest-row-pricing"> <i class="pricing--the-price">  $    $      </i> <span class="pricing--not-the-price">  $    $      </span></div>,
 <div class="rest-row-pricing"> <i class="pricing--the-price">  $    $    $    $  </i> <span class="pricing--not-the-price"> </span></div>,
 <div class="rest-row-pricing"> <i class="pricing--the-price">  $    $    $    </i> <span class="pricing--not-the-price">  $        </span></div>,
 <div class="rest-row-pricing"> <i class="pricing--the-price">  $    $      </i> <span class="pricing--not-the-price">  $    $      </span></div>,
 <div class="rest-row-pricing"> <i class="pricing--the-price">  $    $      </i> <span class="pricing--not-the-price">  $    $      </span></div>,
 <div class="rest-row-pricing"> <i class="pricing--the-price">  $    $      </i> <span class="pricing--not-the-price">  $    $      </span></div>,
 <div class="rest-row-pricing"> <i class="pricing--the-price">  $    $      </i> <span class="pricing--not-the-price">  $    

In [66]:
# print out EACH number of dollar signs per restaurant
# this one is trickier to eliminate the html. Hint: try a nested find
for entry in soup.find_all('div', {'class':'rest-row-pricing'}):
    print(entry.find('i').text, entry.find('i').text.count('$'))

  $    $       2
  $    $    $    $   4
  $    $    $     3
  $    $       2
  $    $       2
  $    $       2
  $    $       2
  $    $    $    $   4
  $    $    $     3
  $    $    $     3
  $    $    $     3
  $    $    $     3
  $    $       2
  $    $    $    $   4
  $    $    $    $   4
  $    $       2
  $    $       2
  $    $    $    $   4
  $    $    $     3
  $    $    $     3
  $    $    $     3
  $    $       2
  $    $       2
  $    $       2
  $    $    $     3
  $    $    $     3
  $    $    $    $   4
  $    $    $    $   4
  $    $    $    $   4
  $    $    $    $   4
  $    $    $     3
  $    $    $    $   4
  $    $       2
  $    $       2
  $    $       2
  $    $    $    $   4
  $    $    $     3
  $    $    $     3
  $    $       2
  $    $    $     3
  $    $       2
  $    $    $     3
  $    $       2
  $    $    $    $   4
  $    $       2
  $    $    $    $   4
  $    $    $    $   4
  $    $    $     3
  $    $       2
  $    $       2
  $    $       2
 

That looks great, but what if I wanted just the number of dollar signs per restaurant? Can you figure out a way to simply print out the number of dollar signs per restaurant listed?

In [ ]:
# print the number of dollars signs per restaurant

That's weird -- an empty set. Did we find the wrong element? What's going on here? Discuss.

How can we debug this? Any ideas?

<a id="selenium"></a>
## Introducing Selenium

Selenium is a headless browser. It allows us to render JavaScript just as a human-navigated browser would.

To install Selenium, use one of the following:
- **Anaconda:** `conda install -c conda-forge selenium`
- **pip:** `pip install selenium`

You will also need GeckoDriver (this assumes you are using Homebrew for Mac): 

- ```brew install geckodriver```

Additionally, you will need to have downloaded the [Firefox browser](https://www.mozilla.org/en-US/firefox/new/?utm_source=google&utm_medium=cpc&utm_campaign=Firefox-Brand-US-GGL-Exact&utm_term=firefox&utm_content=A144_A203_A006336&gclid=Cj0KEQjwnPLKBRC-j7nt1b7OlZwBEiQAv8lMLJUyReT6cPzSYdmEA6uD3YDoieuuuusddgAU7XH6smEaAoje8P8HAQ&gclsrc=aw.ds) for the application in this lesson.

In [ ]:
! pip install selenium

In [1]:
# import
from selenium import webdriver

Selenium requires us to determine a default browser to run. I'm going to opt for Firefox, but Chromium is also a very common choice. http://selenium-python.readthedocs.io/faq.html

In [39]:
# STOP
# what is going to happen when I run the next cell?

In [3]:
# create a driver called Firefox
driver = webdriver.Firefox(executable_path = "C:/Users/13305/Documents/data_science_ga/geckodriver.exe")

Pretty crazy, right? Let's close that driver.

In [4]:
# close it
driver.close()

In [5]:
# let's boot it up, and visit a URL of our choice
driver = webdriver.Firefox(executable_path="C:/Users/13305/Documents/data_science_ga/geckodriver.exe")
driver.get("http://www.python.org")

Awesome. Now we're getting somewhere: programmatically controlling our browser like a human.

Let's return to our problem at hand. We need to visit the OpenTable listing for DC. Once there, we need to get the html to load. In the next cell, prove you can programmatically visit the page.

In [6]:
# visit our OpenTable page
driver = webdriver.Firefox(executable_path="C:/Users/13305/Documents/data_science_ga/geckodriver.exe")
driver.get("http://www.opentable.com/washington-dc-restaurant-listings")

# always good to check we've got the page we think we do
assert "OpenTable" in driver.title

In [7]:
driver.title

'Washington, D.C. Area Restaurants List | OpenTable'

In [8]:
driver.close()

<a id="selenium-js"></a>
### Running JavaScript before scraping

Now, to resolve our JavaScript problem, there's a few things we can do. What I'll do in this case is request that the page load, wait one second, and then I'm going to grab the source html from the page. Because the page should believe I'm visiting from a live connection on a browser client, the JavaScript should render to be a part of the page source. I can then grab the page source.

In [9]:
# import sleep
from time import sleep

In [10]:
# visit our relevant page
driver = webdriver.Firefox(executable_path="C:/Users/13305/Documents/data_science_ga/geckodriver.exe")
driver.get("http://www.opentable.com/washington-dc-restaurant-listings")

# wait one second
sleep(1)

#grab the page source
html = driver.page_source

**Pop Quiz:** What do we need to do with this HTML?

In [36]:
! pip install lxml

In [11]:
import lxml

In [14]:
# BeautifulSoup it!
html = BeautifulSoup(html, "html.parser")

Now, let's return to our earlier problem: How do we locate bookings on the page?

In [15]:
# print out the number bookings for all restaurants
html.find_all('div', {'class':'booking'})

[<div class="booking"><span class="tadpole"></span> Booked 22 times today</div>,
 <div class="booking"><span class="tadpole"></span> Booked 97 times today</div>,
 <div class="booking"><span class="tadpole"></span> Booked 7 times today</div>,
 <div class="booking"><span class="tadpole"></span> Booked 14 times today</div>,
 <div class="booking"><span class="tadpole"></span> Booked 28 times today</div>,
 <div class="booking"><span class="tadpole"></span> Booked 29 times today</div>,
 <div class="booking"><span class="tadpole"></span> Booked 12 times today</div>,
 <div class="booking"><span class="tadpole"></span> Booked 50 times today</div>,
 <div class="booking"><span class="tadpole"></span> Booked 16 times today</div>,
 <div class="booking"><span class="tadpole"></span> Booked 7 times today</div>,
 <div class="booking"><span class="tadpole"></span> Booked 28 times today</div>,
 <div class="booking"><span class="tadpole"></span> Booked 132 times today</div>,
 <div class="booking"><span c

In [16]:
# now print out each booking for the listings using a loop
for entry in html.find_all('div', {'class':'booking'}):
    print(entry)

<div class="booking"><span class="tadpole"></span> Booked 22 times today</div>
<div class="booking"><span class="tadpole"></span> Booked 97 times today</div>
<div class="booking"><span class="tadpole"></span> Booked 7 times today</div>
<div class="booking"><span class="tadpole"></span> Booked 14 times today</div>
<div class="booking"><span class="tadpole"></span> Booked 28 times today</div>
<div class="booking"><span class="tadpole"></span> Booked 29 times today</div>
<div class="booking"><span class="tadpole"></span> Booked 12 times today</div>
<div class="booking"><span class="tadpole"></span> Booked 50 times today</div>
<div class="booking"><span class="tadpole"></span> Booked 16 times today</div>
<div class="booking"><span class="tadpole"></span> Booked 7 times today</div>
<div class="booking"><span class="tadpole"></span> Booked 28 times today</div>
<div class="booking"><span class="tadpole"></span> Booked 132 times today</div>
<div class="booking"><span class="tadpole"></span> Bo

Let's grab just the text of each of these entries.

In [17]:
# do the same as above, but grabbing only the text content
for entry in html.find_all('div', {'class':'booking'}):
    print(entry.text)

 Booked 22 times today
 Booked 97 times today
 Booked 7 times today
 Booked 14 times today
 Booked 28 times today
 Booked 29 times today
 Booked 12 times today
 Booked 50 times today
 Booked 16 times today
 Booked 7 times today
 Booked 28 times today
 Booked 132 times today
 Booked 5 times today
 Booked 70 times today
 Booked 40 times today
 Booked 11 times today
 Booked 13 times today
 Booked 19 times today
 Booked 15 times today
 Booked 16 times today
 Booked 12 times today
 Booked 48 times today
 Booked 11 times today
 Booked 11 times today
 Booked 10 times today
 Booked 17 times today
 Booked 16 times today
 Booked 6 times today
 Booked 3 times today
 Booked 17 times today
 Booked 26 times today
 Booked 3 times today


In [18]:
driver.close()

We've succeeded!

<a id="selenium-regex"></a>
### Using regex to only get digits

But we can clean this up a little bit. We're going to use regular expressions (regex) to grab only the digits that are available in each of the text.

The best way to get good at regex is to, well, just keep trying and testing: http://pythex.org/

In [19]:
# import regex
import re

Given we haven't covered regex, I'll show you how to use the search function to match any given digit.

In [20]:
# for each entry, grab the text
for booking in html.find_all('div', {'class':'booking'}):
    # match all digits
    match = re.search(r'\d+', booking.text)
    
    if match:
        # print if found
        print(match.group())
    else:
        # otherwise pass
        pass

22
97
7
14
28
29
12
50
16
7
28
132
5
70
40
11
13
19
15
16
12
48
11
11
10
17
16
6
3
17
26
3


Before we demonstrate all the other amazing things about headless browsers, let's finish up collecting the data we want from this current example. Do you suppose the html parsing we wrote above will still work on the page source we've grabbed from our headless browser?

To be most efficient, we want to only do a single loop for each entry on the page. That means we want to find what element all of other other elements (name, location, price, bookings) is housed within. Where on the page is each entry located?

In [23]:
# print out all entries
#   NOTE: Has many entries. Uncomment the below code to run it!

soup.find_all('div', {'class':'result content-section-list-row cf with-times'})

[]

Look over the page. Does every single entry have each element we're seeking?
> I did this previously. I know for a fact that not every element has a number of recent bookings. That's probably exactly why OpenTable houses this in JavaScript: they want to continously update the number of bookings with the most relevant number of values.

In [24]:
# We want to only retrieve the text of the bookings.
# But what would happen if we just naively print the text of each node?

for entry in html.find_all('div', {'class':'result content-section-list-row cf with-times'}):
    print(entry.find('div', {'class':'booking'}))   # try adding .text

In [25]:
# if we find the element we want, we print it. Otherwise, we print 'ZERO'
for entry in html.find_all('div', {'class':'result content-section-list-row cf with-times'}):
        divs = entry.find('div', {'class':'booking'})
        
        if divs:
            print(divs.text)
        else:
            print('ZERO')

What do you notice takes the place when booking is not found?

We could use exception handling (`try`/`except` blocks) to resolve this. However, exceptions should only be used to handle rare or unexpected errors -- never for normal program flow.

In this case, we expect that some entries will be zero. So, we can just use an `if` statement that tests whether there are any `divs` present; if not, display `'ZERO'`. Here's a demo:

In [26]:
# if we find the element we want, we print it. Otherwise, we print 'ZERO'
for entry in html.find_all('div', {'class':'result content-section-list-row cf with-times'}):
    booking_tag = entry.find('div', {'class':'booking'})
    
    if booking_tag:
        print(booking_tag.text)
    else:
        print('ZERO')

After previously completing this, we observed that all other elements WILL be returned. This means we do not have to always handle these cases.

<a id="challenge-pandas"></a>
### Challenge: Use Pandas to create a DataFrame of bookings

However, the onus is on you to now put all the pieces together.

Loop through each entry. For each entry, grab the relevant information we want (name, location, price, bookings). Produce a dataframe with the columns "name","location","price","bookings" that contains the 100 entries we would like.

In [29]:
import pandas as pd

In [30]:
# I'm going to create my empty df first
dc_eats = pd.DataFrame(columns=["name","location","price","bookings"])

In [27]:
#A: 
for entry in html.find_all('div', {'class':'result content-section-list-row cf with-times'}):
    # grab the name
    entry.find(name='span', attrs={'class':'rest-row-name-text'}).text
    # grab the location
    entry.find(name='span', attrs={'class':'rest-row-meta--location rest-row-meta-text'}).text
    # grab the pricing
    entry.find('div', {'class':'rest-row-pricing'}).find('i').text.count('$')
    # try to find the number of bookings:
    bookings = 'NA'
    booking_tag = entry.find('div', {'class':'booking'})
    if booking_tag:
        match=re.search('\d+',booking_tag.text)
        if match:
            bookings = match.group()
    dc_eats.loc[len(dc_eats)]=[name, location, price, bookings]
    

In [31]:
# check out our work
dc_eats.head()

,name,location,price,bookings


Awesome! We succeeded.

<a id="selenium-typing"></a>
### Auto-typing using Selenium

Now, let's explore some of the other functionality of a webdriver. We've barely scratched the surface.

In [32]:
# we can send keys as well
# import
from selenium.webdriver.common.keys import Keys

In [34]:
# open Firefox
driver = webdriver.Firefox(executable_path="C:/Users/13305/Documents/data_science_ga/geckodriver.exe")
# visit Python
driver.get("http://www.python.org")
# verify we're in the right place
assert "Python" in driver.title

Let's try automatedly typing `pycon` in the search box and hitting the return key:

In [35]:
# find the search position
elem = driver.find_element_by_name("q")

# clear it
elem.clear()

# type in pycon
elem.send_keys("pycon")

# send those keys
elem.send_keys(Keys.RETURN)

In [36]:
# close
driver.close()

In [37]:
# all at once:
driver = webdriver.Firefox(executable_path="C:/Users/13305/Documents/data_science_ga/geckodriver.exe")
driver.get("http://www.python.org")
assert "Python" in driver.title

elem = driver.find_element_by_name("q")
elem.clear()
elem.send_keys("pycon")
elem.send_keys(Keys.RETURN)
#assert "No results found." not in driver.page_source
driver.close()

The above example (and many others) are available in the Selenium docs: http://selenium-python.readthedocs.io/getting-started.html

What is especially important is exploring functionality like locating elements: http://selenium-python.readthedocs.io/locating-elements.html#locating-elements

FAQ:
http://selenium-python.readthedocs.io/faq.html

### Summary

In this lesson, we used the Beautiful Soup library to locate elements on a website then scrape their text. We also used the Selenium headless browser to run JavaScript first before retrieving the page contents.

## Class Exercise
- For all CL housing listings on the front page, scrape the price and title and any other attributes into a dataframe

### Summary

In this lesson, we used the Beautiful Soup library to locate elements on a website then scrape their text. We also used the Selenium headless browser to run JavaScript first before retrieving the page contents.